<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

Данный проект направлен на построение модели, которая сможет отфильтровывать комменарии с неподходящей, для платформы, лексикой. 

## Подготовка

Устанавливаем необходимые библиотеки.

In [1]:
import sys
!{sys.executable} -m pip install spacy
!{sys.executable} -m spacy download en

⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     |████████████████████████████████| 13.9 MB 2.4 MB/s eta 0:00:01    |▉                               | 348 kB 2.4 MB/s eta 0:00:06
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
import re
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
import numpy as np
import re
import nltk
from sklearn.datasets import load_files
from pymystem3 import Mystem
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import f1_score
from sklearn.dummy import DummyRegressor
from nltk.stem import WordNetLemmatizer 

import pickle
from nltk.corpus import stopwords

from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC
import spacy





Скачиваем данные

In [3]:
data = pd.read_csv("/datasets/toxic_comments.csv");

Порверяем на вылеты, пустые данные, некорректные данные

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [5]:
data.head(5)

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [6]:
data.shape

(159292, 3)

In [7]:
data.isnull().sum()

Unnamed: 0    0
text          0
toxic         0
dtype: int64

In [8]:
data = data.drop_duplicates()
data.shape

(159292, 3)

In [9]:
data['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

Разница положительных и отрицательных маркеров около 9 раз. 

Данные обработаны корректно

## Обучение

# Лематизация текста/ Создание признаков

Делим данные на тестовые и тренировачные данные

In [10]:
def clean(text):
    
    text = text.lower()    
    text = re.sub(r"(?:\n|\r)", " ", text)
    text = re.sub(r"[^a-zA-Z ]+", "", text).strip()
    
    return text

data['text'] = data['text'].apply(clean)
display(data.head())

,Unnamed: 0,text,toxic
0,0,explanation why the edits made under my userna...,0
1,1,daww he matches this background colour im seem...,0
2,2,hey man im really not trying to edit war its j...,0
3,3,more i cant make any real suggestions on impro...,0
4,4,you sir are my hero any chance you remember wh...,0


In [11]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

def lemmatize(text):
    temp = []
    for token in nlp(text):
        if token.is_stop == False:
            temp.append(token.lemma_)
    return " ".join(temp)


tqdm.pandas()
data['text'] = data['text'].progress_apply(lemmatize)

100%|██████████| 159292/159292 [17:48<00:00, 149.14it/s]


In [12]:
data_train,data_test= train_test_split(data, test_size=0.1, random_state=0)




In [13]:
corpus_train = data_train['text'].values
corpus_test = data_test['text'].values


In [14]:
print(corpus_train[1:5])

['replace flip flop flipflop text page move flipflip need delete redirect direction case     aug   utc'
 'yearbook greek orthodox church link website verifiable source'
 'well luck time   degenerate foodfight not   sad repeat conflict proleft proisrael   enemy enemy thing get mess common cause progressive nazis   sad state affair   suppose easy   disagree action state israel respect palestinian issue grandson nazi not feel right publicly criticise   not holocaust guilt   matter accept legacy grandfather accept measure responsibility action   pointless fight people like slim deserving people fight     god work   thing   want moderate private email conversationsit sound like plan offesive importantly sound like tiff sam spade felonious monk   btw feel free ignore not clue      apr   utc'
 'relevence coase therom understand policy implication externalatie new topic find hard coase helpful understand policy implication externality help']


In [15]:
stopwords = set(stopwords.words('english'))

Приводим данные к необходимому виду для обработки моделью

In [16]:
count_tf_idf = TfidfVectorizer(stop_words = stopwords)

In [17]:
corpus_train = count_tf_idf.fit_transform(corpus_train)
corpus_test = count_tf_idf.transform(corpus_test)

In [18]:
test_features = data_test.drop(['toxic'], axis=1)
test_target =data_test['toxic']

# Обучение моделей

Возьмем для обучения 2 модели - LogisticRegression, LinearSVC

In [19]:
parameters = {'C': np.linspace(5, 15, num = 3, endpoint = True),
             'max_iter': [1000]}
lrm = LogisticRegression()
clf = GridSearchCV(lrm, parameters,
                  cv=5,
                  scoring='f1',
                  n_jobs=-1,
                  verbose=2)
clf.fit(corpus_train, data_train['toxic'])


Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV] END ...............................C=5.0, max_iter=1000; total time= 2.0min
[CV] END ...............................C=5.0, max_iter=1000; total time= 1.9min
[CV] END ...............................C=5.0, max_iter=1000; total time= 1.4min
[CV] END ...............................C=5.0, max_iter=1000; total time= 1.7min
[CV] END ...............................C=5.0, max_iter=1000; total time= 1.9min
[CV] END ..............................C=10.0, max_iter=1000; total time= 1.8min
[CV] END ..............................C=10.0, max_iter=1000; total time= 2.0min
[CV] END ..............................C=10.0, max_iter=1000; total time= 2.3min
[CV] END ..............................C=10.0, max_iter=1000; total time= 2.2min
[CV] END ..............................C=10.0, max_iter=1000; total time= 2.6min
[CV] END ..............................C=15.0, max_iter=1000; total time= 2.7min
[CV] END ..............................C=15.0, ma

GridSearchCV(cv=5, estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': array([ 5., 10., 15.]), 'max_iter': [1000]},
             scoring='f1', verbose=2)

In [20]:
print(f"Наилучший показатель f1 на кросс-валидации : {clf.best_score_:.3f}")
print(clf.best_params_)

Наилучший показатель f1 на кросс-валидации : 0.769
{'C': 10.0, 'max_iter': 1000}


In [21]:
parameters = {'C': np.linspace(5, 15, num = 7, endpoint = True)}
lsvcm = LinearSVC(max_iter = 3000)
clf_lsvc = GridSearchCV(lsvcm, parameters,
                  cv=5,
                  scoring='f1',
                  n_jobs=-1,
                  verbose=2)

In [22]:
clf_lsvc.fit(corpus_train, data_train['toxic'])

Fitting 5 folds for each of 7 candidates, totalling 35 fits
[CV] END ..............................................C=5.0; total time=   3.1s
[CV] END ..............................................C=5.0; total time=   3.1s
[CV] END ..............................................C=5.0; total time=   2.9s
[CV] END ..............................................C=5.0; total time=   2.5s
[CV] END ..............................................C=5.0; total time=   2.5s
[CV] END ................................C=6.666666666666667; total time=   3.3s
[CV] END ................................C=6.666666666666667; total time=   3.4s
[CV] END ................................C=6.666666666666667; total time=   3.2s
[CV] END ................................C=6.666666666666667; total time=   3.2s
[CV] END ................................C=6.666666666666667; total time=   3.2s
[CV] END ................................C=8.333333333333334; total time=   4.1s
[CV] END ................................C=8.3333

GridSearchCV(cv=5, estimator=LinearSVC(max_iter=3000), n_jobs=-1,
             param_grid={'C': array([ 5.        ,  6.66666667,  8.33333333, 10.        , 11.66666667,
       13.33333333, 15.        ])},
             scoring='f1', verbose=2)

In [23]:
print(f"Наилучший показатель f1 на кросс-валидации : {clf_lsvc.best_score_:.3f}")

Наилучший показатель f1 на кросс-валидации : 0.762


Выбор лучшей модели по метрике F1 (чем больше значение тем модель эффективнее):

In [24]:
print(f"Наилучший показатель f1 на кросс-валидации : {clf.best_score_:.3f}")
print(f"Наилучший показатель f1 на кросс-валидации : {clf_lsvc.best_score_:.3f}")

Наилучший показатель f1 на кросс-валидации : 0.769
Наилучший показатель f1 на кросс-валидации : 0.762


Лучше себя показала модель  - LogisticRegression

Проверим ее на тестовой выборке

In [25]:
clf_lsvc = clf_lsvc.best_estimator_

In [26]:
predict = clf_lsvc.predict(corpus_test)
f1_lr = f1_score(test_target, predict)
print(f1_lr)

0.7564102564102565


Модель проверена на тестовых данных

## Выводы

Основная цель проекта - создать модель, которая бы определяля комментраии, которые содержат некоректную лексику. 
1. Были установлены необходимые библиотеки
2. Были скачены и проверены данные. 
3. Было проверено 2 модели LogisticRegression и LinearSVC - на кросс валидации луший результат показала LogisticRegression
